# --> Importations

In [44]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# ToDoList

- Preprocess CSV file
- Preprocess images
- Build CNN model for images trained on labels
- Make the CNN model return a probability vector
- Use a MLP model to add table features with this probabilty vector
- Tune both models

# Infos

- I will start with simple model and preprocessing, not trying to create visual border around the disease
- I won't use some of the table features at the beginning

# Global variables

In [45]:
DATA_ENTRY_CSV_PATH = "/kaggle/input/data/Data_Entry_2017.csv"
#LABELS
#N_LABELS
#ONE_HOT_LABELS

# Preprocess CSV file

In [46]:
data_entry_df = pd.read_csv(DATA_ENTRY_CSV_PATH)
data_entry_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


In [47]:
data_entry_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Follow-up #,112120.0,8.573751,15.406320,0.000,0.000,3.000,10.000,183.0000
Patient ID,112120.0,14346.381743,8403.876972,1.000,7310.750,13993.000,20673.000,30805.0000
Patient Age,112120.0,46.901463,16.839923,1.000,35.000,49.000,59.000,414.0000
OriginalImage[Width,112120.0,2646.078844,341.246429,1143.000,2500.000,2518.000,2992.000,3827.0000
Height],112120.0,2486.438842,401.268227,966.000,2048.000,2544.000,2991.000,4715.0000
OriginalImagePixelSpacing[x,112120.0,0.155649,0.016174,0.115,0.143,0.143,0.168,0.1988
y],112120.0,0.155649,0.016174,0.115,0.143,0.143,0.168,0.1988
Unnamed: 11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# --> Get labels and number of labels

In [48]:
LABELS = []
for label in data_entry_df["Finding Labels"].unique():
    for word in label.split("|"):
        if word not in LABELS:
            LABELS.append(word)
N_LABELS = len(LABELS)
print("Number of labels :", N_LABELS)
print("Labels : ", LABELS)

Number of labels : 15
Labels :  ['Cardiomegaly', 'Emphysema', 'Effusion', 'No Finding', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']


# --> One hot encoded labels

In [49]:
n_label_list = np.arange(N_LABELS)
ONE_HOT_LABELS = []
for index, label in enumerate(LABELS):
    one_hot_label = []
    for i in range(N_LABELS):
        if n_label_list[index] == i:
            one_hot_label.append(1)
        else:
            one_hot_label.append(0)
    print(one_hot_label)
    ONE_HOT_LABELS.append(one_hot_label)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


# --> Dropping some columns to make it simpler

In [50]:
data_entry_df.drop(columns=["Patient ID", "Follow-up #", "Unnamed: 11"], inplace=True) #Some of those features are important but they need some feature engeneering so they will be implemented later

In [51]:
data_entry_df

,Image Index,Finding Labels,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,39,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,29,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,30,F,PA,2048,2500,0.168,0.168


# Preprocess images for the CNN model 

# --> Make image dataset from directories

In [88]:
images_array = []
for image_dir in os.listdir("../input/data"):
    if "images_" in image_dir:
        images_dir_path = os.path.join("../input/data", image_dir, "images")
        for image_name in os.listdir(images_dir_path):
            images_array += np.array(Image.open(os.path.join("../input/data", image_dir, "images", image_name)))
print(images_array)

NameError: name 'Image' is not defined